Convert TDT and EDF data to NWB

July 15 2024

Noah Markowitz

In [1]:
import glob
import os.path as op
import os
import numpy as np
import pandas as pd
import re

from datetime import datetime
from uuid import uuid4
from dateutil.tz import tzlocal

from uuid import uuid4
from hdmf.common.table import DynamicTable
from hdmf.backends.hdf5.h5_utils import H5DataIO
from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.file import Subject, ElectrodeTable
from pynwb.device import Device
from pynwb.ecephys import ElectrodeGroup, ElectricalSeries
import h5py

In [2]:
from ieeg2nwb import IEEG2NWB, load_nwb_settings

# Setup

Specify info on subject and study

In [3]:
# Path to study directory
# NOTE: You should not read in the original study saved on the server.
#       In reading files there's a chance for corruption. Make a copy of it
# Path to w drive is: /run/user/1000/gvfs/smb-share:server=sykisilon1,share=mehtalab/
study_directory = "/media/hbml/HDD2/natus_api/data/natus/amayareyes-jacinto_IC-Day-5A"

# Freesurfer info
freesurfer_subject_directory = "/home/hbml/freesurfer_7.1.1/subjects"
freesurfer_subject_id = "NS144_02"

# NWB filename. Suggested to follow the BIDS template such as: sub-NS001_ses-implant01_desc-day1_ieeg.nwb
nwb_filename = "/media/hbml/HDD2/natus_api/data/nwb/sub-NS144_ses-implant02_desc-ImplantDay5_ieeg.nwb"

# Text fields for NWB file. Leave blank for default to be filled

# Typically the freesurfer and subject ID are the same but not always such as if they had multiple implants

subject_info = {
    "subject_id": "NS144",
    "age": 30, # An integer or ISO 8601 format (ex: 30 years old = "P30Y"). ISO may be good if children to also have months
    "sex": "U", # Options: [M,F,U]
    "species": "Homo sapiens",
    "description": "intracranial patient" # can put another description if you'd like
}

notes = """

"""

session_description = """

"""

session_id = ""


experiment_description = """

"""

experimenter = """

"""

institution = """

"""

lab = """

"""

data_collection = """

"""

# Options: DC1-DC14, TRIG, OSAT, PR, PLETH
other_chans = []

# Channels that contain the primary eeg data
# For TDT should be the stores such as: RawX, EEG1, EEG2
eeg_data_stores = ["EEG1", "EEG2"]

# Analog acquisitions containing data not necessarily eeg
# For each analog store make a dictionary as below then 
# add each dictionary to the list `analog_acquisitions`
analog_data_dict = {
    "name": "",
    "description": "",
    "store": "",
    "comments": "",
    "channels": [0,1,2],
    "externalize": False
}

analog_acquisitions = []

# Digital acquisitions
digital_data_dict = {
    "name": "",
    "description": "",
    "store": "",
    "comments": "",
    "channels": ["PtC2","PtC4"],
    "externalize": False
}
digital_acquisitions = []

# Read in settings

In [6]:
settings = load_nwb_settings()

# Create IEEG2NWB instance

In [ ]:
converter = IEEG2NWB()

# Meta-data, Subject

Fill in meta-data if some are empty

In [ ]:
# Fill this dictionary. It will be used for conversion
nwb_info = {
    "notes": notes,
    "session_description": session_description,
    "experiment_description": experiment_description,
    "institution": institution,
    "lab": lab,
    "data_collection": data_collection,
    "session_id": session_id,
}

In [7]:
meta_data_settings = settings["meta_data"]
for param, value in nwb_info.items():
    if isinstance(value, datetime):
        setattr(converter, param, value)
    else:
        setattr(
            converter,
            param,
            value if len(value.strip())!=0 else meta_data_settings[param]
        )

Create subject object

In [8]:
subject_settings = settings["subject"]
for param, value in subject_settings.items():
    if isinstance(subject_info[param], int):
        continue
    if len(subject_info[param])==0:
        subject_info[param] = subject_settings[param]

subject = converter.create_subject(**subject_info)

# Read raw data

In [ ]:
converter.read_input(study_directory, eeg_chans=eeg_data_stores, make_device=True)

# Read correspondence sheet, make table, create eeg acquisitions

In [10]:
# Import correspondence sheet
corr_list = glob.glob(op.join(freesurfer_subject_directory, freesurfer_subject_id, "elec_recon", "*correspondence*"))
corr_file = corr_list[0]

# Read it
converter.read_labelfile(corr_file)

# Clean it
converter.clean_labelfile()

# Create electrode groups
converter.create_electrodeGroups()

# Convert correspondence sheet to table
converter.labelfile2table()

# Create table regions
converter.create_table_regions()

# Each region corresponds to a different ElectricalSeries. Create every ES
converter.regions2es()

# Add other TimeSeries if specified

In [ ]:
analog_data_dict = {
    "name": "",
    "description": "",
    "store": "",
    "comments": "",
    "channels": [0,1,2],
    "externalize": False
}

for a in analog_acquisitions:
    pass

# Parse events

Must have the following:
* An events table as a pandas DataFrame instance. Must contain
    - start_time column
    - end_time column
* A `dynamic_columns` dictionary variable with name and description of all columns

# Create NwbFile object then write

In [ ]:
converter.write_nwb(nwb_filename)